# Operacje na danych

Ten notebook zawiera gotowe do zastosowania przekształcenia danych, które możecie Państwo sobie skopiować do swoich projektów. Operacje będą prowadzone na dwóch zbiorach danych z Państwa książki:

1. Customers
2. Products

Poszczególne operacje będą zgrupowane w sekcjach, a w każej sekcji - typowe "problemy" i rozwiążania tych problemów

# 0. Ładowanie bibliotek

Biblioteki importujemy zazwyczaj przez nadawanie im aliasów od pierwszych liter. To jest dobra praktyka :)

In [2]:
import pandas as pd
import numpy as np

# 1. Wczytywanie danych

### Jak wczytać dane z pliku csv?

In [12]:
customers = pd.DataFrame.from_csv("./customers.txt", sep="\t", index_col=None)
products = pd.DataFrame.from_csv("./products.txt", sep="\t", index_col=None)

In [13]:
customers.head(10)

,customerid,householdid,gender,firstname
0,174596,53949999,M,DANIEL
1,68239,49927024,M,JIM
2,10203,20914414,F,DONNA
3,174288,53957047,F,DIONE
4,68099,49927024,M,JIM
5,174257,53949983,F,TRISHA
6,173506,53476818,M,DANTE
7,51886,20359142,F,ANTHONY
8,143249,20287377,F,NANCY
9,174650,53958894,M,GREG


In [14]:
products.head(10)

,PRODUCTID,PRODUCTNAME,PRODUCTGROUPCODE,PRODUCTGROUPNAME,INSTOCKFLAG,FULLPRICE
0,10001,NaN,CA,CALENDAR,N,15
1,10002,NaN,CA,CALENDAR,N,10
2,10003,NaN,CA,CALENDAR,N,10
3,10004,NaN,BK,BOOK,Y,90
4,10005,NaN,BK,BOOK,Y,16
5,10006,NaN,BK,BOOK,N,10
6,10007,NaN,BK,BOOK,N,7
7,10008,NaN,BK,BOOK,N,11
8,10009,NaN,BK,BOOK,N,10
9,10010,NaN,BK,BOOK,N,19


# 2. Selekcja danych

### Jak mogę wybrać dane w oparciu o jakieś pojedyncze kryterium?

In [39]:
# Sposób 1: wektor binarny tak/nie (warunek spełniony/niespełniony) na bazie zapytania:

kategoria_ca = products.PRODUCTGROUPCODE == "CA"
kategoria_ca.head(4)

0     True
1     True
2     True
3    False
Name: PRODUCTGROUPCODE, dtype: bool

In [40]:
products.loc[kategoria_ca, :].head(5)

,PRODUCTID,PRODUCTNAME,PRODUCTGROUPCODE,PRODUCTGROUPNAME,INSTOCKFLAG,FULLPRICE,price+10,cena_razy_10
0,10001,NaN,CA,CALENDAR,N,15,25,150
1,10002,NaN,CA,CALENDAR,N,10,20,100
2,10003,NaN,CA,CALENDAR,N,10,20,100
837,10838,NaN,CA,CALENDAR,Y,11,21,110
861,10862,NaN,CA,CALENDAR,N,39,49,390


In [43]:
# Sposób 2: zapytaniem pseudo-sql:

products.query("PRODUCTGROUPCODE == 'CA'").head(4)

,PRODUCTID,PRODUCTNAME,PRODUCTGROUPCODE,PRODUCTGROUPNAME,INSTOCKFLAG,FULLPRICE,price+10,cena_razy_10
0,10001,NaN,CA,CALENDAR,N,15,25,150
1,10002,NaN,CA,CALENDAR,N,10,20,100
2,10003,NaN,CA,CALENDAR,N,10,20,100
837,10838,NaN,CA,CALENDAR,Y,11,21,110


### Jak mogę wybrać dane na podstawie kilku kryteriów?

Tak samo jak wyżej, tylko podając dwa warunki i łącząc je operatorem logicznym :D Nazwiasy oddzielają od siebie poszczególne polecenie

In [45]:
# Sposób 1: 

kategoria_ca_lub_bk = (products.PRODUCTGROUPCODE == "CA") | (products.PRODUCTGROUPCODE == "BK")
kategoria_ca_lub_bk.head(4)

0    True
1    True
2    True
3    True
Name: PRODUCTGROUPCODE, dtype: bool

In [46]:
products.loc[kategoria_ca_lub_bk, :].head(4)

,PRODUCTID,PRODUCTNAME,PRODUCTGROUPCODE,PRODUCTGROUPNAME,INSTOCKFLAG,FULLPRICE,price+10,cena_razy_10
0,10001,NaN,CA,CALENDAR,N,15,25,150
1,10002,NaN,CA,CALENDAR,N,10,20,100
2,10003,NaN,CA,CALENDAR,N,10,20,100
3,10004,NaN,BK,BOOK,Y,90,100,900


In [49]:
# Sposób 2:
products.query("PRODUCTGROUPCODE == 'CA' | PRODUCTGROUPCODE == 'BK' ").head(4)

,PRODUCTID,PRODUCTNAME,PRODUCTGROUPCODE,PRODUCTGROUPNAME,INSTOCKFLAG,FULLPRICE,price+10,cena_razy_10
0,10001,NaN,CA,CALENDAR,N,15,25,150
1,10002,NaN,CA,CALENDAR,N,10,20,100
2,10003,NaN,CA,CALENDAR,N,10,20,100
3,10004,NaN,BK,BOOK,Y,90,100,900


# 3. Agregacja danych

### Jak mogę grupować dane według jakiegoś kryterium i policzyć?

In [18]:
# Sposób 1:

products.groupby("PRODUCTGROUPCODE").size()

PRODUCTGROUPCODE
#N       1
AP      86
AR    3300
BK     240
CA      31
FR      25
GA     231
OC      71
OT      55
dtype: int64

In [17]:
# Sposób 2:
pd.value_counts(products.PRODUCTGROUPCODE)

AR    3300
BK     240
GA     231
AP      86
OC      71
OT      55
CA      31
FR      25
#N       1
Name: PRODUCTGROUPCODE, dtype: int64

### Jak można podsumować zawartość kolumny numerycznej w danej grupie albo kilku kolumn? Potrzebuję średniej, wariancji, odchylenia, itp? Innymi słowy - kilka miar do jednej kolumny w grupowaniu

In [21]:
products.groupby("PRODUCTGROUPCODE").agg({
    'FULLPRICE': [np.mean, np.std, np.sum], # średnia, odchylenie i suma kolumny fullprice
    'INSTOCKFLAG': [pd.value_counts]       # liczność grup dla kolumny INSTOCKFLAG, dla danej grupy produktów [wart. Y, wart. N]
    
})

FULLPRICE                       INSTOCKFLAG
                        mean         std      sum value_counts
PRODUCTGROUPCODE                                              
#N                  0.000000         NaN        0            1
AP                 23.348837    9.827653     2008     [43, 43]
AR                432.698485  363.453337  1427905   [3209, 91]
BK                 40.937500  231.353522     9825   [135, 105]
CA                 40.032258  109.974386     1241     [16, 15]
FR                  0.000000    0.000000        0      [20, 5]
GA                 11.268398    9.822193     2603   [117, 114]
OC                 24.154930   10.644446     1715      [66, 5]
OT                 22.054545   22.970527     1213     [44, 11]

### Jak mogę napisać swoją własną funkcję grupującą?

Funkcja przyjmuje postać:


```python

lambda wektor_wartości: funkcja(wektor_wartości)

```

Gdzie funkcja na wektorze wartości zwraca pojedynczą wartość.

Przykład (idiotyczny): funkcja dodająca wartość 10 do wszystkich elementów wekora cen i sumująca je

In [22]:
products.groupby("PRODUCTGROUPCODE").agg({
    'FULLPRICE': lambda vec: np.sum(vec+10)
})

,FULLPRICE
PRODUCTGROUPCODE,
#N,10
AP,2868
AR,1460905
BK,12225
CA,1551
FR,250
GA,4913
OC,2425
OT,1763


Bardziej sensowny przykład: najczęściej pojawiająca się wartość wśród kolumny INSTOCKFLAG w ramach groupy produktów:

In [25]:
products.groupby("PRODUCTGROUPCODE").agg({
    'FULLPRICE': lambda vec: np.argmax(pd.value_counts(products.INSTOCKFLAG))
})

,FULLPRICE
PRODUCTGROUPCODE,
#N,Y
AP,Y
AR,Y
BK,Y
CA,Y
FR,Y
GA,Y
OC,Y
OT,Y


# 4. Kolumny wyliczeniowe

Wszystkie operacje na kolumnach są **zwektoryzowane**, czyli operacje odnoszą się do całego wektora:

In [30]:
(products.FULLPRICE + 10).head(10) # dodanie 10 do każdego elementu wektora cen

0     25
1     20
2     20
3    100
4     26
5     20
6     17
7     21
8     20
9     29
Name: FULLPRICE, dtype: int64

### Jak mogę wyliczyć coś w oparciu o kolumnę i od razu dodać ją do data frame?

In [36]:
# Sposób 1: na piechotkę

new_col = products.FULLPRICE + 10
products['price+10'] = new_col
products.head(5)

,PRODUCTID,PRODUCTNAME,PRODUCTGROUPCODE,PRODUCTGROUPNAME,INSTOCKFLAG,FULLPRICE,price+10,cena_razy_10
0,10001,NaN,CA,CALENDAR,N,15,25,150
1,10002,NaN,CA,CALENDAR,N,10,20,100
2,10003,NaN,CA,CALENDAR,N,10,20,100
3,10004,NaN,BK,BOOK,Y,90,100,900
4,10005,NaN,BK,BOOK,Y,16,26,160


In [37]:
# Sposób 2: nowy operator assign

products = products.assign(cena_razy_10 = lambda dane: dane.FULLPRICE * 10)
products.head(5)

,PRODUCTID,PRODUCTNAME,PRODUCTGROUPCODE,PRODUCTGROUPNAME,INSTOCKFLAG,FULLPRICE,price+10,cena_razy_10
0,10001,NaN,CA,CALENDAR,N,15,25,150
1,10002,NaN,CA,CALENDAR,N,10,20,100
2,10003,NaN,CA,CALENDAR,N,10,20,100
3,10004,NaN,BK,BOOK,Y,90,100,900
4,10005,NaN,BK,BOOK,Y,16,26,160


# 5. Jak łączyć dane

In [53]:
df1 = pd.DataFrame({
    'klucz': ['a', 'a', 'b', 'b', 'c'],
    'wartosc': [1, 2, 10, 20, 100]
})

df2 = pd.DataFrame({
    'klucz': ['a', 'a', 'b', 'b', 'd'],
    'wartosc': [3, 4, 30, 40, 1000]
})

Joiny w SQL-u

**INNER JOIN -  łączymy tylko pasujące klucze**

In [58]:
df1.merge(df2, on="klucz", how="inner")

,klucz,wartosc_x,wartosc_y
0,a,1,3
1,a,1,4
2,a,2,3
3,a,2,4
4,b,10,30
5,b,10,40
6,b,20,30
7,b,20,40


**LEFT JOIN - zachowujemy całą lewą tabelę i dołączamy tą z prawej**

In [59]:
df1.merge(df2, on="klucz", how="left")

,klucz,wartosc_x,wartosc_y
0,a,1,3.0
1,a,1,4.0
2,a,2,3.0
3,a,2,4.0
4,b,10,30.0
5,b,10,40.0
6,b,20,30.0
7,b,20,40.0
8,c,100,NaN


**RIGHT JOIN - zachowujemy całą prawą tabelę i dołączamy tą z lewej**

In [60]:
df1.merge(df2, on="klucz", how="right")

,klucz,wartosc_x,wartosc_y
0,a,1.0,3
1,a,2.0,3
2,a,1.0,4
3,a,2.0,4
4,b,10.0,30
5,b,20.0,30
6,b,10.0,40
7,b,20.0,40
8,d,NaN,1000
